In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
import geopy
import geopandas

In [68]:
arrests = pd.read_csv('Arrests.csv')

In [4]:
locator = geopy.Nominatim(user_agent="myGeocoder")
location = locator.geocode("Jefferson Park Ave, Charlottesville, VA")

In [22]:
print(location.latitude, location.longitude)

38.0255412 -78.5149948


In [5]:
arrests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17380 entries, 0 to 17379
Data columns (total 12 columns):
ArrestID              17380 non-null int64
ArrestDatetime        17380 non-null object
FirstName             17378 non-null object
HouseNumber           15450 non-null float64
LastName              17380 non-null object
MiddleName            16236 non-null object
NameSuffix            1610 non-null object
Statute               17362 non-null object
StatuteDescription    17362 non-null object
Street                16497 non-null object
Race                  17363 non-null object
Sex                   17372 non-null object
dtypes: float64(1), int64(1), object(10)
memory usage: 1.6+ MB


In [4]:
arrests.head()

,ArrestID,ArrestDatetime,FirstName,HouseNumber,LastName,MiddleName,NameSuffix,Statute,StatuteDescription,Street,Race,Sex
0,1001,2019-06-13T07:19:00.000Z,CHRISTOPHER,1505.0,STANLEY,DEAN,NaN,18.2-415,Disturbing the peace,UNIVERSITY AVE,Black,Female
1,1002,2019-06-13T06:59:00.000Z,GABRIEL,109.0,SANTOS,J,NaN,18.2-388,PROFANE SWEARING OR INTOXICATION IN PUBLIC,CULBRETH RD,White,Male
2,1003,2019-06-13T04:00:00.000Z,CAROLYN,350.0,RICE,PAIGE,NaN,18.2-111,Embezzlement,PARK ST,White,Female
3,1004,2019-06-13T00:02:00.000Z,MICHAEL,7051.0,COOPER,JEROME,NaN,19.2-306,PROBATION VIOLATION - TYPE NOT CLEAR FROM RECORD,LOUISA RD,Black,Male
4,1005,2019-06-12T18:15:00.000Z,Roy,606.0,Correll,Chapman,NaN,18.2-137(B)(i),"MONUMENT: INTENTIONAL DAMAGE, VALUE <$1000",E MARKET ST,White,Male


To convert the date, we will split it into two columns. One will be a float64 in units of months (do we ignore the year?). The other will be a float64 in units of hours.

In [1]:
def null_values(df, return_table=True):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * mis_val / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(4)
    print("There are", len(df.columns), "columns in total.")
    print("There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
    if return_table:
        return mis_val_table_ren_columns

The only nonmissing features are the ID, the date and time, and the last name.

In [5]:
miss_values = null_values(arrests)
pd.set_option('display.max_rows', None)
miss_values

There are 12 columns in total.
There are 9 columns that have missing values.


,Missing Values,% of Total Values
NameSuffix,15770,90.7365
HouseNumber,1930,11.1047
MiddleName,1144,6.5823
Street,883,5.0806
Statute,18,0.1036
StatuteDescription,18,0.1036
Race,17,0.0978
Sex,8,0.0460
FirstName,2,0.0115


To impute missing data in 'Sex', all nan values should be replaced with 'Unknown'.

In [7]:
arrests['Sex'].unique()

array(['Female', 'Male', nan, 'Unknown'], dtype=object)

The two rows without first names are interesting. In the row with ID 2262, the last name is a placeholder 'JOHN DOE'.

In [23]:
arrests.loc[arrests['FirstName'] != arrests['FirstName']]

,ArrestID,ArrestDatetime,FirstName,HouseNumber,LastName,MiddleName,NameSuffix,Statute,StatuteDescription,Street,Race,Sex
696,1697,2019-03-15T00:38:00.000Z,NaN,NaN,PURPLERAIN,NaN,NaN,905,DRUNKENESS,BROWN ST,Black,Female
5261,2262,2018-12-24T13:06:00.000Z,NaN,1131.0,JOHN DOE,NaN,NaN,18.2-388,PROFANE SWEARING OR INTOXICATION IN PUBLIC,5TH ST SW,White,Male


What are 'nwt', 'doc', and 'jpg' doing in the 'Race' feature?

In [ ]:
arrests.loc[arrests['Race'] == 'nwt']

In [ ]:
arrests.loc[arrests['Race'] == 'doc']

We see that there are multiple descriptions that correspond to the same statute. Therefore, we drop the StatuteDescription feature because it does not add anything to the data.

If we keep the names, it is reasonable to perform feature engineering that assigns a unique number to a full name, including the suffix if applicable, since we see that one person can and often commits more than one offense. We will have to convert every name to uppercase, since the capitalization is inconsistent.

There are 472 unique statutes. A OneHotEncoder will result in the number of features increasing to about 480.

In [24]:
arrests['NameSuffix'].unique()

array([nan, 'Jr', 'II', 'III', 'Sr', 'IV'], dtype=object)

In [ ]:
arrests['StatuteDescription'].value_counts()

In [ ]:
arrests['Statute'].value_counts()

In [19]:
arrests['Statute'].nunique()

472

## Data cleaning

We drop the ArrestID and StatuteDescription because the former is useless and the latter is redundant with Statute. In fact, there are many descriptions that can potentially refer to the same statute. We also drop the names since there are too many unique names for an analysis to be useful with them kept.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [99]:
arrests = pd.read_csv('Arrests.csv')

In [100]:
useless_and_redundant = ['ArrestID', 'StatuteDescription']
names = ['FirstName', 'LastName', 'MiddleName', 'NameSuffix']
arrests.drop(useless_and_redundant, axis=1, inplace=True)
arrests.drop(names, axis=1, inplace=True)

In [101]:
arrests['Year'] = pd.to_datetime(arrests['ArrestDatetime']).apply(
    lambda x: float(x.strftime('%Y')) if x == x else np.nan)
arrests['Month'] = pd.to_datetime(arrests['ArrestDatetime']).apply(
    lambda x: float(x.strftime('%m')) if x == x else np.nan) 
arrests['Day'] = pd.to_datetime(arrests['ArrestDatetime']).apply(
    lambda x: float(x.strftime('%d')) if x == x else np.nan) 
arrests['Time'] = pd.to_datetime(arrests['ArrestDatetime']).apply(
    lambda x: 3600 * float(x.strftime('%H')) + 60 * float(x.strftime('%M')) + float(x.strftime('%S'))
    if x == x else np.nan)

In [102]:
def convert_housenumber(x):
    if x == x:
        return str(int(x))
    else:
        return

arrests['HouseNumber'] = arrests['HouseNumber'].apply(convert_housenumber)
arrests['Address'] = arrests['HouseNumber'] + ' ' + arrests['Street'] + ', CHARLOTTESVILLE, VA'
arrests.drop(['ArrestDatetime', 'HouseNumber', 'Street'], axis=1, inplace=True)

In [103]:
arrests.head()

,Statute,Race,Sex,Year,Month,Day,Time,Address
0,18.2-415,Black,Female,2019.0,6.0,13.0,26340.0,"1505 UNIVERSITY AVE, CHARLOTTESVILLE, VA"
1,18.2-388,White,Male,2019.0,6.0,13.0,25140.0,"109 CULBRETH RD, CHARLOTTESVILLE, VA"
2,18.2-111,White,Female,2019.0,6.0,13.0,14400.0,"350 PARK ST, CHARLOTTESVILLE, VA"
3,19.2-306,Black,Male,2019.0,6.0,13.0,120.0,"7051 LOUISA RD, CHARLOTTESVILLE, VA"
4,18.2-137(B)(i),White,Male,2019.0,6.0,12.0,65700.0,"606 E MARKET ST, CHARLOTTESVILLE, VA"


We will use LabelEncoding for 'Sex' and OneHotEncoding for 'Race'.

In [108]:
def convert_races(x):
    if x == 'nwt' or x == 'doc' or x == 'jpg' or x != x:
        return 'Unknown'
    else:
        return x
    
arrests['Race'] = arrests['Race'].apply(convert_races)

In [112]:
pd.set_option('display.max_rows', None)
arrests['Race'].unique()

array(['Black', 'White', 'Asian or Pacific Islander', 'Unknown',
       'American Indian/Alaskan Native'], dtype=object)